In [29]:
import pandas as pd
import math
import logging
from sklearn.model_selection import TimeSeriesSplit
from copy import deepcopy
from prophet import Prophet
from prophet.serialize import model_to_json


from eval import my_grid_search_cv

logging.getLogger("prophet").setLevel(logging.CRITICAL)
logging.getLogger("cmdstanpy").setLevel(logging.CRITICAL)

logging.getLogger("fbprophet.forecaster").propagate = False

# from tqdm import tqdm
# tqdm.pandas()

In [30]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

'/root/restaurants'

In [31]:
air_visit_data = pd.read_csv("data/raw/air_visit_data.csv")
data = deepcopy(air_visit_data)
data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [32]:
date_info = pd.read_csv("data/raw/date_info.csv")
date_info = date_info.drop(columns=["day_of_week"])
date_info.head()

,calendar_date,holiday_flg
0,2016-01-01,1
1,2016-01-02,1
2,2016-01-03,1
3,2016-01-04,0
4,2016-01-05,0


In [33]:
holidays = (
    date_info[date_info["holiday_flg"] == 1]
    .rename(columns={"holiday_flg": "holiday", "calendar_date": "ds"})
    .reset_index(drop=True)
)
holidays = holidays[["holiday", "ds"]]
holidays["holiday"] = "holiday"
holidays.head()

,holiday,ds
0,holiday,2016-01-01
1,holiday,2016-01-02
2,holiday,2016-01-03
3,holiday,2016-01-11
4,holiday,2016-02-11


In [ ]:
# def fit_for_each_store(directory, model, data):

from tqdm.auto import tqdm

tqdm.pandas()

small = []

directory = "models/prophet_for_each_store_with_holidays"
os.makedirs(directory, exist_ok=True)

models = dict()

param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.1, 0.5],
    "seasonality_prior_scale": [0.01, 0.1, 1, 10],
    "seasonality_mode": ["additive", "multiplicative"],
    "holidays": [holidays],
}

for store_id, store_data in tqdm(data.groupby("air_store_id")):

    if store_id == "air_17bed6dbf7c8b0fc":
        param_grid = {
            "changepoint_prior_scale": [0.001, 0.1, 0.5],
            "seasonality_prior_scale": [0.01, 0.1, 1, 10],
            "seasonality_mode": ["additive", "multiplicative"],
            "holidays": [holidays],
        }
    
    print(store_id, store_data.shape[0])

    file_path = os.path.join(directory, f"model_{store_id}.json")

    if os.path.exists(file_path):
        continue

    X = store_data[["visit_date"]].reset_index(drop=True)
    y = store_data[["visitors"]].reset_index(drop=True)

    n_splits = max(min(math.trunc((store_data.shape[0] - 5) / 30), 5), 2)

    cv = TimeSeriesSplit(n_splits=n_splits, gap=3, test_size=30)

    if store_data.shape[0] <= 65:
        small.append(store_id)
        cv = TimeSeriesSplit(
            n_splits=2, gap=1, test_size=int(store_data.shape[0] * 0.2)
        )

    params = my_grid_search_cv(Prophet, X, y, param_grid, cv=cv, n_jobs=5)

    models[store_id] = Prophet(**params)

    df = pd.concat([X, y], axis=1)
    df = df.rename(columns={"visit_date": "ds", "visitors": "y"})
    models[store_id].fit(df)

    with open(file_path, "w") as fout:
        fout.write(model_to_json(models[store_id]))

In [43]:
from prophet.serialize import model_from_json


def get_models(directory_path):
    models = dict()

    for file in os.listdir(directory_path):
        with open(f"{directory_path}/{file}", "r") as fin:
            store_id = file[6:]
            store_id = store_id[:-5]
            models[store_id] = model_from_json(fin.read())

    return models

In [44]:
models = get_models("models/prophet_for_each_store_with_holidays")

In [45]:
# from tqdm import tqdm
from tqdm.auto import tqdm

submission = pd.read_csv("data/raw/sample_submission.csv")
submission = submission.rename(columns={"visit_date": "ds"})


def make_prediction(row):
    store_id = row["id"][:20]
    date = pd.DataFrame([row["id"][21:]])
    date = date.rename(columns={0: "ds"})

    pred = models[store_id].predict(date)
    pred = pred[["yhat"]]
    pred[pred < 0] = 0

    row["visitors"] = pred["yhat"][0]
    return row


tqdm.pandas()
submission = submission.progress_apply(make_prediction, axis=1)

  0%|          | 0/32019 [00:00<?, ?it/s]

In [46]:
directory = "data/submissions"
os.makedirs(directory, exist_ok=True)

submission.to_csv(directory + "/prophet_for_each_store_with_holidays.csv", index=False)